<a href="https://colab.research.google.com/github/srishmaalladi/Katha-vachak/blob/main/test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install unsloth
!pip install pyngrok
!pip install flask_cors
!pip install pillow
!pip install transformers
!pip install torch
!pip install sumy
!pip install nltk

In [2]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
from unsloth import FastLanguageModel
model,tokenizer=FastLanguageModel.from_pretrained('/content/drive/MyDrive/final_model')


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2024.11.11: Fast Llama patching. Transformers:4.46.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 7.5. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Unsloth 2024.11.11 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [4]:
from flask import Flask, request, jsonify
from pyngrok import ngrok
from unsloth import FastLanguageModel
from PIL import Image
from transformers import TextStreamer, AutoTokenizer, AutoModelForCausalLM
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.text_rank import TextRankSummarizer
import torch
from flask_cors import CORS
import re
import base64
import requests
import io
import nltk

nltk.download('punkt_tab')

# Initialize Flask app
app = Flask(__name__)
CORS(app)
PORT = 5000

# Set up ngrok for public access
ngrok.set_auth_token("2nTaRxHGdDYre0DMbc02Do1TzPz_5dYfMsAZ8bfMidRijWmY7")
public_url = ngrok.connect(PORT)  # Automatically generates a random public URL

# Print the public URL for external access
print(f"To access the global link, please click {public_url}")

# Load the model and tokenizer
story_keywords = ["story", "generate", "tale", "narrative", "adventure"]

def contains_story_keywords(user_input):
    """Check for any story-related keywords in the user input"""
    return any(keyword in user_input.lower() for keyword in story_keywords)

def split_into_sentences(text):
    """Split text into sentences using regular expressions"""
    sentences = re.split(r'(?<=[.!?])\s+', text.strip())
    return sentences

def group_sentences_into_scenes(sentences, group_size=4):
    """Group sentences into scenes with specified group size (4 by default) and include scene numbers."""
    scenes = [
        {"scene_no": i // group_size + 1, "scene_text": ' '.join(sentences[i:i+group_size])}
        for i in range(0, len(sentences), group_size)
    ]
    print(f"group_sentences_into_scenes: {scenes}")
    return scenes


def summarize_text_with_sumy(text, compression_ratio):
    """
    Summarize the given text using Sumy TextRank and compression ratio.
    """
    parser = PlaintextParser.from_string(text, Tokenizer("english"))
    summarizer = TextRankSummarizer()

    # Calculate the number of sentences for the summary
    num_sentences = max(1, int(len(parser.document.sentences) * compression_ratio))
    summary = summarizer(parser.document, num_sentences)

    # Combine the sentences into a single string
    summarized_text = " ".join(str(sentence) for sentence in summary)
    return summarized_text

@app.route("/", methods=["POST"])
def generate_story():
    user_input = request.json.get("input", "") # Get user input from the POST request
    word_limit=request.json.get("word_limit","")
    print(word_limit)
    # Check if the input contains story-related keywords
    if contains_story_keywords(user_input):
        # Prepare inputs for the model (direct input from the user)
        inputs = tokenizer([user_input], return_tensors="pt").to("cuda")

        # Enable native 2x faster inference using FastLanguageModel
        FastLanguageModel.for_inference(model)  # This line is essential for performance

        # Generate the story
        generated_story = model.generate(**inputs)
        # //print(generated_story)

        # Decode the generated tokens to text
        story_text = tokenizer.decode(generated_story[0], skip_special_tokens=True)
        story_text = story_text.replace("\n", " ")
        while user_input in story_text:
            story_text = story_text.replace(user_input, "").strip()
        total_words = len(story_text.split())
        if word_limit and total_words > word_limit:
          compression_ratio = word_limit / total_words
          story_text = summarize_text_with_sumy(story_text, compression_ratio)



        sentences = split_into_sentences(story_text)
        scenes = group_sentences_into_scenes(sentences)

        # Return the story with images in the response
        print(scenes)
        return jsonify(scenes)
    else:
        # User input does not contain story-related keywords; provide an alternative response
        return jsonify({"message": "I'm trained to generate stories. Your input doesn't seem to ask for one."})

# Run the Flask app
if __name__ == "__main__":
    app.run(port=PORT)



[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


To access the global link, please click NgrokTunnel: "https://87bf-34-16-231-10.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


300


INFO:werkzeug:127.0.0.1 - - [27/Nov/2024 09:28:26] "POST / HTTP/1.1" 200 -


group_sentences_into_scenes: [{'scene_no': 1, 'scene_text': "It tells the story of Walter White, a high school chemistry teacher who turns to manufacturing and selling methamphetamine with his former student, Jesse Pinkman, in order to secure his family's financial future after being diagnosed with terminal lung cancer. It has been described as one of the greatest television series of all time and has won numerous awards, including 16 Primetime Emmy Awards, 58 Emmy nominations, two Golden Globe Awards, and 58 Golden Globe nominations. Breaking Bad is known for its complex and realistic portrayal of the criminal underworld, as well as its exploration of themes such as morality, family, and the consequences of one's actions. The show is also known for its use of unexpected twists and turns, as well as its intense and often violent scenes."}, {'scene_no': 2, 'scene_text': "As the series progresses, Walter and Jesse's business grows and they become involved in the criminal underworld of Al